# Moter dataset Anomaly detection

In [4]:
dataset = 'data/sensor_data_b.csv'

### Check dataset proportions

In [5]:
import pandas as pd 

df = pd.read_csv(dataset, header=None) 
label_counts = df.iloc[:, -1].value_counts()

print(label_counts)


NameError: name 'dataset' is not defined

### dataset split 

In [2]:
label_0_data = df[df.iloc[:, -1] == 0]
label_1_data = df[df.iloc[:, -1] == 1]
label_2_data = df[df.iloc[:, -1] == 2]

label_0_data.to_csv('data/label_0_data.csv', index=False)
label_1_data.to_csv('data/label_1_data.csv', index=False)
label_1_data.to_csv('data/label_2_data.csv', index=False)

NameError: name 'df' is not defined

To detect anomalies, we plan to input data into the model at 1-second intervals with a sampling rate of 100Hz. Therefore, we will segment the data into 70 samples for each 1-second intervals.

In [3]:
import pandas as pd

label_0 = pd.read_csv("data/label_0_data.csv")
label_1 = pd.read_csv("data/label_1b_data.csv")
label_2 = pd.read_csv("data/label_2_data.csv")
label_3 = pd.read_csv("data/label_3_data.csv")

In [4]:
len_0 = len(label_0)
len_1 = len(label_1)
len_2 = len(label_2)
len_3 = len(label_3)

label_X = [label_0, label_1, label_2, label_3]
rem = [len_0, len_1, len_2, len_3]

for i in range(4):
    r = rem[i] % 100
    label_X[i] = label_X[i].iloc[:rem[i] - r]

    print(len(label_X[i]))



359900
359800
359900
359900


In [5]:
import numpy as np

In [6]:
data_split_0 = np.array_split(label_X[0], len(label_X[0]) // 100)
data_split_1 = np.array_split(label_X[1], len(label_X[1]) // 100)
data_split_2 = np.array_split(label_X[2], len(label_X[2]) // 100)
data_split_3 = np.array_split(label_X[3], len(label_X[3]) // 100)


y_label_0 = np.zeros(len(data_split_0))
y_label_1 = np.ones(len(data_split_1))
y_label_2 = np.full(len(data_split_2), 2)
y_label_3 = np.full(len(data_split_3), 3)


### Split train, validataion, test set

train_ratio = 0.7
validation ratio = 0.15
test ratio = 0.15

In [59]:
train_ratio = 0.7
temp_ratio = 0.3
val_ratio = 0.5
test_ratio = temp_ratio / 2

data_splits = [data_split_0, data_split_1, data_split_2, data_split_3]
y_data_splits = [y_label_0, y_label_1, y_label_2, y_label_3]

train_sizes = []
train_data = []
test_data = []

y_train_data = []
y_test_data = []


for i in range(4):
    data_len = len(data_splits[i])
    train_size = int(data_len * train_ratio)
    train_sizes.append(train_size)
    train_data.append(data_splits[i][:train_size])
    test_data.append(data_splits[i][train_size:])

    y_train_data.append(y_data_splits[i][:train_size])
    y_test_data.append(y_data_splits[i][train_size:])


# check
for i in range(4):
    print(len(train_data[i]) + len(test_data[i]))

x_train_concat = np.concatenate(train_data, axis = 0)
x_test_concat = np.concatenate(test_data, axis = 0)
y_train_concat = np.concatenate(y_train_data, axis=0)
y_test_concat = np.concatenate(y_test_data, axis=0)


val_size = int(x_test_concat.shape[0] * val_ratio)
test_size = x_test_concat.shape[0] - val_size

# validation & test set split
x_val = x_test_concat[:val_size]
x_test = x_test_concat[val_size:]

y_val = y_test_concat[:val_size]
y_test = y_test_concat[val_size:]

# check
print(f"x_val size: {x_val.shape[0]}")
print(f"x_test size: {x_test.shape[0]}")
print(f"y_val size: {y_val.shape[0]}")
print(f"y_test size: {y_test.shape[0]}")

# check 
print(x_train_concat.shape[0] + x_test.shape[0] + x_val.shape[0])
print(y_train_concat.shape[0] + y_test.shape[0] + y_val.shape[0])

3599
3598
3599
3599
x_val size: 2160
x_test size: 2160
y_val size: 2160
y_test size: 2160
14395
14395


In [13]:
x_train_concat.shape

(10075, 100, 3)

In [14]:
y_train_concat.shape

(10075,)

In [19]:
from keras.utils import to_categorical

# Convert y_train and y_valid to one-hot encoding
y_train_one_hot = to_categorical(y_train_concat, num_classes=4)  # Assuming classes are 0, 1, 2, 3
y_valid_one_hot = to_categorical(y_val, num_classes=4)  # Assuming you have a similar y_valid
y_test_one_hot = to_categorical(y_test, num_classes=4)  # Assuming you have a similar y_test

# Check the shape
print(y_train_one_hot.shape)  # Should be (10070, 4)
print(y_valid_one_hot.shape) 

(10075, 4)
(2160, 4)


### Define the Model Architecture

In [20]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15294558232718816693
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 11378904359985974365
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 6933955910558672802
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:1"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 15573965249812017374
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:2"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 13699355476082648183
physical_device_desc: "device: XLA_GPU device"
]


In [8]:
import tensorflow as tf
# from tensorflow.python.client import device_lib
# print(tf.__version__)
# print(device_lib.list_local_devices())


In [21]:
import tensorflow as tf
import keras
from keras.layers import LSTM, Dropout, Dense, InputLayer
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam


In [22]:
model = keras.models.Sequential([
    # Conv Block 1 
    keras.layers.Conv1D(filters=32, kernel_size=3, strides=1, padding="same", activation="relu",
                        input_shape=[100, 3]),
    keras.layers.Conv1D(filters=32, kernel_size=3, strides=1, padding="same", activation="relu"), 

    # Pooling layer
    keras.layers.MaxPooling1D(pool_size=2),

    # Conv Block 2 
    keras.layers.Conv1D(filters=64, kernel_size=3, strides=1, padding="same", activation="relu"),
    keras.layers.Conv1D(filters=64, kernel_size=3, strides=1, padding="same", activation="relu"),           
    
    # Pooling layer
    keras.layers.MaxPooling1D(pool_size=2),  

    # Conv Block 3
    keras.layers.Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"),
    keras.layers.Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"),   
    
    # Pooling layer        
    keras.layers.MaxPooling1D(pool_size=2),  

    # Conv Block 4
    keras.layers.Conv1D(filters=256, kernel_size=3, strides=1, padding="same", activation="relu"),

    keras.layers.GlobalAveragePooling1D(),

    # Updated output layer for multi-class classification
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(4, activation="softmax")  # Change to 4 units and softmax activation
])

# Compile the model
model.compile(loss="categorical_crossentropy",  # Use categorical crossentropy
              optimizer="adam",
              metrics=["accuracy"])  # Accuracy metric is still appropriate

# Train the model
history = model.fit(
    x_train_concat, y_train_one_hot,  # Ensure y_train is one-hot encoded for multi-class
    epochs=20,  
    batch_size=32,
    validation_data=(x_val, y_valid_one_hot),
    verbose=1,
    # class_weight=class_weights_dict,
    # callbacks=[early_stop]
)

# Get predictions
y_pred = model.predict(x_val)
y_pred_classes = np.argmax(y_pred, axis=1)  # Get the class with the highest probability

# # Save the model
# model.save("u_net3.h5")


Train on 10075 samples, validate on 2160 samples
Epoch 1/20
10075/10075 [==============================] - 7s 714us/step - loss: 0.4110 - accuracy: 0.8461 - val_loss: 0.0023 - val_accuracy: 0.9995
Epoch 2/20
10075/10075 [==============================] - 7s 711us/step - loss: 2.5682e-04 - accuracy: 0.9999 - val_loss: 0.0015 - val_accuracy: 0.9995
Epoch 3/20
10075/10075 [==============================] - 7s 709us/step - loss: 5.3878e-06 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 0.9995
Epoch 4/20
10075/10075 [==============================] - 7s 708us/step - loss: 2.1092e-06 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 0.9995
Epoch 5/20
10075/10075 [==============================] - 7s 708us/step - loss: 9.8702e-07 - accuracy: 1.0000 - val_loss: 0.0016 - val_accuracy: 0.9995
Epoch 6/20
10075/10075 [==============================] - 7s 707us/step - loss: 5.4053e-07 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 0.9995
Epoch 7/20
10075/10075 [===================

In [60]:
loss, accuracy = model.evaluate(x_test, y_test_one_hot, verbose=1)
print(loss)
print(accuracy)


2160/2160 [==============================] - 0s 195us/step
1.4681439769283468e-05
1.0


In [61]:
y_pred = model.predict(x_test, verbose=1)

print(y_pred[:10])
print(y_test[:10])

2160/2160 [==============================] - 0s 186us/step
[[5.1180602e-09 8.2138288e-16 1.0000000e+00 3.1631824e-08]
 [4.1516701e-09 9.1624189e-16 1.0000000e+00 3.9026578e-08]
 [3.2488654e-09 1.1299241e-15 1.0000000e+00 5.4546273e-08]
 [8.1576204e-09 6.7346388e-16 1.0000000e+00 2.1231500e-08]
 [4.0178705e-09 1.0710237e-15 1.0000000e+00 4.5767720e-08]
 [4.0133288e-09 1.0020468e-15 1.0000000e+00 4.3624741e-08]
 [2.8052942e-09 1.1997001e-15 9.9999988e-01 6.3734568e-08]
 [2.4682896e-09 1.4465514e-15 9.9999988e-01 8.1635385e-08]
 [5.4364686e-09 8.5498320e-16 1.0000000e+00 3.1969368e-08]
 [1.8718289e-09 1.9389294e-15 9.9999988e-01 1.2549620e-07]]
[2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]


In [39]:
classes = np.unique(y_test)  # 클래스 인덱스 추출
conf_matrix = np.zeros((len(classes), len(classes)), dtype=int)  # 혼동 행렬 초기화

# 길이 확인
print("y_test 길이:", len(y_test))
print("y_pred 길이:", len(y_pred))
y_pred = np.argmax(y_pred, axis=1)  
print(y_pred[0])
y_test = y_test.astype(int)
y_pred = y_pred.astype(int)


# 혼동 행렬 계산
for i in range(len(y_test)):
    conf_matrix[y_test[i], y_pred[i]] += 1

print("혼동 행렬:\n", conf_matrix)b



y_test 길이: 2160
y_pred 길이: 2160
2


IndexError: index 2 is out of bounds for axis 0 with size 2

In [62]:
import numpy as np

# y_pred의 형태 확인 및 정수형으로 변환
if y_pred.ndim == 2:
    y_pred = np.argmax(y_pred, axis=1)
elif y_pred.ndim == 1:
    print("y_pred는 이미 클래스 인덱스입니다.")

# y_test 및 y_pred의 고유 클래스 추출
unique_classes = np.unique(np.concatenate((y_test, y_pred)))  # 두 배열의 고유 클래스 합치기

# 클래스 인덱스를 0부터 시작하도록 조정
class_to_index = {cls: idx for idx, cls in enumerate(unique_classes)}
y_test_mapped = np.array([class_to_index[cls] for cls in y_test])
y_pred_mapped = np.array([class_to_index[cls] for cls in y_pred])


print(len(y_test_mapped))
print(len(y_pred_mapped))
# 혼동 행렬 초기화
num_classes = len(unique_classes)
conf_matrix = np.zeros((num_classes, num_classes), dtype=int)

# 혼동 행렬 계산
for i in range(len(y_test)):
    conf_matrix[y_test_mapped[i], y_pred_mapped[i]] += 1

# 혼동 행렬 출력
print("혼동 행렬:\n", conf_matrix)


2160
2160
혼동 행렬:
 [[1080    0]
 [   0 1080]]
